deep research就是 planning + execute + synthesize

In [ ]:
#model init
import os
from dotenv import load_dotenv
load_dotenv()

from langchain.chat_models import init_chat_model

qwen_plus_model = init_chat_model(
    "qwen-plus",
    model_provider="openai",
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    temperature=0.5
)


In [53]:
#response_format

from pydantic import  BaseModel,Field

class WebSearchItem(BaseModel):
    """网页搜索项"""
    query:str = Field(...,description = "用于网页搜索的关键词，只写一个关键词")
    reason: str = Field(...,description="该搜索词对回答原问题的必要说明")

class WebSearchPlan(BaseModel):
    """网页搜索计划（包含多条搜索指令）"""
    searchs: list[WebSearchItem] = Field(...,description="为精确回答原问题需执行的网页搜索列表")

In [54]:
#planning agent
from langchain.agents import create_agent

system_prompt = """
    你是一个专业的研究助手，需基于用户查询完成以下核心任务，确保搜索的精准高效：
    1.设计10个不重复的网页搜索关键词，关键词需贴合查询核心需求，兼顾精准度与覆盖性；
    2.为每个关键词单独说明搜索原因，说明该关键词如何助理解答用户查询，原因需具体可落地，不笼统。
"""

plainning_agent = create_agent(
    model = qwen_plus_model,
    system_prompt = system_prompt,
    response_format = WebSearchPlan
)   

In [55]:
response = plainning_agent.invoke(
    {
        "messages":[
            {"role":"user","content":"AI在教育中的应用"}
        ]
    }
)
search_plan = response["structured_response"]
queries = [item for item in search_plan.searchs]
queries

[WebSearchItem(query='AI教育应用案例', reason='通过具体案例了解AI在教育中的实际应用场景，如智能辅导、个性化学习等，帮助理解AI如何改变传统教学模式'),
 WebSearchItem(query='人工智能课堂教学应用', reason='聚焦AI在课堂环境中的使用，例如语音识别、智能白板、自动答疑系统，有助于分析其对师生互动的影响'),
 WebSearchItem(query='AI自适应学习系统', reason='自适应学习是AI教育的核心方向之一，搜索该关键词可获取技术原理及主流平台信息，如Knewton、猿题库等'),
 WebSearchItem(query='AI批改作业系统', reason='了解AI在作业批改中的应用现状，包括作文自动评分、编程作业判题等，评估其准确性和普及程度'),
 WebSearchItem(query='教育机器人应用', reason='教育机器人是AI的实体化体现，搜索此词可获取其在儿童编程、语言学习等场景的应用实例'),
 WebSearchItem(query='AI个性化推荐学习内容', reason='探究AI如何根据学生行为数据推荐学习资源，提升学习效率，理解算法在教育中的适配性'),
 WebSearchItem(query='AI教育政策发展', reason='了解国家或地区对AI教育的支持政策与发展规划，判断其推广背景与未来趋势'),
 WebSearchItem(query='AI在在线教育平台的应用', reason='主流在线教育平台（如Coursera、网易云课堂）如何集成AI功能，分析其在课程推荐、学习路径规划中的作用'),
 WebSearchItem(query='AI教育伦理问题', reason='探讨AI在教育中可能引发的数据隐私、算法偏见等问题，全面评估其应用风险'),
 WebSearchItem(query='AI教师辅助工具', reason='了解AI如何帮助教师备课、学情分析、课堂管理，提升教学效率，反映人机协同的教学新模式')]

In [56]:
#execute agent 
#execute agent 的作用是联网搜索+总结

from langchain_community.chat_models import ChatTongyi

qwen_search_model = ChatTongyi(
    model='qwen-max',
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    model_kwargs={
        "enable_search": True
    }
)

execute_prompt = """
    你是一名专业的研究助理，给定一个关键词后，需通过网络来搜索该关键词，并生成简洁的搜索结果摘要。
    摘要需包含2-3个段落，字数控制在500字以内，需涵盖核心要点。
    表述需简洁凝练，无需使用完整的句子或注重语法规范。
    该摘要将要供他人整合报告使用，因此务必提炼核心信息、剔除无关信息。除摘要本身外，不得添加任何额外评论。
"""

execute_agent = create_agent(
    model = qwen_search_model,
    system_prompt = execute_prompt
)

execute_agent_response = execute_agent.invoke(
    {
        "messages":[{"role":"user","content":"ai教育政策与发展趋势"}]
    }
)
search_content = execute_agent_response["messages"][-1].content
search_content

'2026年教育部将出台专项政策文件，旨在系统部署人工智能在教育领域的应用与推进，构建面向未来的教育体系。政策核心在于根据不同教育阶段特点分类推进AI通识教育，而非仅仅开设编程课程，目标是培养能够主导智能时代的高素质人才。北京市已率先布局，计划完善大中小学AI通识课程体系，并推动高校实验室向中小学开放，为全国政策落地提供示范样本。\n\n技术层面，教育专用大模型实现了规模化突破，国家平台上线了“AI试验场”，汇聚14个AI智能工具及多款智能体，如“育小苗”智能体辅助师生高效获取资源，思政教育智能体覆盖7大应用场景并在高校试用，进一步支持AI教育的场景化落地。需求方面，随着高考报名人数增加以及本科录取率下降，升学压力促使精准化教育需求升级；同时，“二胎”生育小高峰带来的中高考黄金十年、公考报名人数攀升等趋势，也为AI+教育提供了广阔市场空间。产业端已经进入规模化落地期，AI技术正重塑校内外教育教学全场景，包括自动评阅、理化实验测评、精准教学等环节，有效提升了教学效率与质量。'

In [57]:
#synthesize agent
#synthesize的作用是进行汇总

synthesize_prompt = """
    你是一名资深研究员，负责为一项研究查询撰写结构连贯的报告。
    你将收到原始查询需求以及研究助理完成的初步研究成果。
    请首先制定报告大纲，明确报告的结构框架与逻辑脉络。
    随后基于大纲撰写完整报告，并将其作为最终输出。
    内容要有结构有段落，最底层的结果不要用1. 或者 - 分点描述，直接用文字描述。
    最终 报告需要采用markdown格式，内容需详尽深入。
    字数不少于2000字。
"""

class SynthesizeData(BaseModel):
    """研究结果的简短总结"""
    short_summary:str = Field(...,description="研究结果的简短总结")
    '''最终报告'''
    markdown_report: str = Field(...,description="最终报告")
    '''建议进一步研究的主题'''
    follow_up_question: list[str] = Field(...,description="建议进一步研究的主题")
    

In [58]:
from langchain.agents.middleware import SummarizationMiddleware

summary_prompt = '''
    你是一名资深研究员，负责为一项研究查询撰写结构连贯的报告。
    目前有一份报告，你需要在不要过多删减的情况下从现有报告中精简提炼重点。
'''

synthesize_agent = create_agent(
    model = qwen_plus_model,
    system_prompt = synthesize_prompt,
    middleware = [
        SummarizationMiddleware(
            model = qwen_plus_model,
            trigger = ("tokens",20000),
            summary_prompt = summary_prompt
        )
    ],
    response_format = SynthesizeData
)

synthesize_agent_response =  synthesize_agent.invoke(
    {
        "messages":[
            {"role":"user","content":search_content}
        ]
    }
)

In [ ]:
from rich.markdown import Markdown
synthesize_agent_response_summary = synthesize_agent_response["structured_response"].short_summary
synthesize_agent_response_report = synthesize_agent_response["structured_response"].markdown_report
synthesize_agent_response_flow_up_questions = synthesize_agent_response["structured_response"].follow_up_question

clean_markdown = synthesize_agent_response_report.strip("```").strip("markdown").strip()
display(Markdown(clean_markdown))

In [72]:
class deepResearchManager:
    def init(self):
        print("init")

    def run(self,query:str)->None:
        print("Starting reasearch...")
        #Step1：根据用户query生成检索计划
        search_plan = self.plan_search(query)
        #step2:根据生成检索计划进行网页搜索
        search_results = self.perform_searchs(search_plan)
        #step3: 根据检索内容生成markdown格式的报告
        markdown_content,follow_up_question = self.write_report(query,search_results)
        #step4:处理返回结果
        display(Markdown(markdown_content))

        follow_up_question_text = "\n".join(follow_up_question)
        display(Markdown(follow_up_question_text))

        self.save_report_as_md(query,markdown_content)

    def plan_search(self,query:str)->WebSearchPlan:
        print("planning searchs...")
        response = plainning_agent.invoke(
            {"messages":[{"role":"user","content":query}]}
        )
        return response

    def perform_searchs(self,search_plan:WebSearchPlan)->list[str]:
        print("search internet...")
        num_completed = 0
        search_plans = search_plan["structured_response"]
        tasks = [item.query for item in search_plans.searchs]
        results = []
        for task in tasks:
            search_agent_res = execute_agent.invoke(
                {
                    "messages":[{"role":"user","content":task}]
                }
            )
            search_content = search_agent_res["messages"][-1].content
            if search_content is not None:
                results.append(search_content)
            num_completed+=1
            print(f"searching...{num_completed}/{len(tasks)} completed")

        return results

    def write_report(self,query:str,search_results:list[str])->SynthesizeData:
        print("Thinking about report...")
        write_report_query = f"用户的需求是：{query}\n汇总的搜索结果如下:{search_results}"

        synthesize_agent_response = synthesize_agent.invoke(
            {
                "messages":[{"role":"user","content":write_report_query}]
            }
        )
        synthesize_agent_response_summary = synthesize_agent_response["structured_response"].short_summary
        synthesize_agent_response_report = synthesize_agent_response["structured_response"].markdown_report
        synthesize_agent_response_flow_up_questions = synthesize_agent_response["structured_response"].follow_up_question

        clean_markdown = synthesize_agent_response_report.strip("```").strip("markdown").strip()
        return clean_markdown,synthesize_agent_response_flow_up_questions

    def save_report_as_md(self,query:str,markdown_content:str)->None:
        """
        保存生成的报告为markdown文件
        """
        #使用用户的查询作为文件名
        file_query = query.replace(" ","_").replace("：","").replace("?","")

        file_name = f"关于{file_query}的调研报告.md"
        file_path = os.path.join(os.getcwd(),file_name)
        with open(file_path,"w",encoding="utf-8") as file:
            file.write(markdown_content)
        print(f"Report saved as: {file_name}")
        

In [73]:
manager = deepResearchManager()
manager.run("AI在教育中的应用")

Starting reasearch...
planning searchs...
search internet...
searching...1/10 completed
searching...2/10 completed
searching...3/10 completed
searching...4/10 completed
searching...5/10 completed
searching...6/10 completed
searching...7/10 completed
searching...8/10 completed
searching...9/10 completed
searching...10/10 completed
Thinking about report...


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                                     AI在教育中的应用：现状、技术与未来展望                                      ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛


                                                     报告大纲                                                      

 1 引言：人工智能重塑教育格局                                                                                      
    • 教育数字化转型背景                                                                                           
    • AI技术介入教育的核心动因                                                                                     
 2 AI在教育中的主要应用场景                                                                                        
    • 智能辅导与个性化学习系统                                                                                     
    • 自动化评估与作业批改技术                                                                                     
    • 虚拟教师与沉浸式教学体验                                                                                     
    • 教育机器人与课堂辅助实践                                                                                     
    • 教师辅助决策与教学优化支持                                                                                   
 3 核心技术支撑体系                                                                                                
    • 机器学习与自适应算法                                                                                         
    • 自然语言处理与语音识别                                                                                       
    • 大数据与学习分析                                                                                             
    • 情感计算与人机交互技术                                                                                       
 4 AI教育应用的成效与实证研究                                                                                      
    • 学习成效提升的证据支持                                                                                       
    • 教学效率与资源可及性的改善                                                                                   
    • 不同学科领域的差异化影响                                                                                     
 5 面临的挑战与伦理问题                                                                                            
    • 数据隐私与信息安全风险                                                                                       
    • 算法偏见与教育公平性争议                                                                                     
    • 教师角色的重构与职业发展焦虑                                                                                 
    • 技术成本与基础设施不均                                                                                       
 6 政策导向与国家战略布局                                                                                          
    • 中国AI+教育顶层设计进展                                                                                      
    • 2026年专项政策预期内容                                                                                       
    • 师资培训与智能教育新基建                                                                                     
 7 未来发展趋势与展望                                                                                              
    • 智能化、个性化与集成化融合方向                                                                               
    • 全学段AI通识教育推进路径                                                                                     
    • 构建安全可信的AI教育生态系统                                                                                 
 8 结论                                                                                                            

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

                                            引言：人工智能重塑教育格局                   

AI如何在特殊教育中发挥作用？ 国际上有哪些成功的AI教育政策案例可以借鉴？ 如何评估AI教育工具的实际教学效果？         
AI是否会最终取代人类教师？其边界在哪里？ 在边远地区推广AI教育面临哪些具体障碍及应对策略？

Report saved as: 关于AI在教育中的应用的调研报告.md
